<a href="https://colab.research.google.com/github/ShuoHengLi/PDFComparator/blob/main/PDFComparator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 基本工具下載



In [ ]:
!pip install langchain-community huggingface_hub
!pip install langchain pypdf sentence_transformers chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 85.7 MB/s eta 0:00:

# 讀取外部PDF檔

In [ ]:
pdf1_url = "https://www.planetebook.com/free-ebooks/anna-karenina.pdf"
pdf2_url = "https://www.planetebook.com/free-ebooks/war-and-peace.pdf"
pdf1_path = "Book1.pdf"
pdf2_path = "Book2.pdf"

import requests
def download_pdf(url, filename):
    """Downloads a PDF from a URL to a local file."""
    response = requests.get(url)
    response.raise_for_status()
    with open(filename, 'wb') as f:
        f.write(response.content)
    return filename

# Download the PDFs
download_pdf(pdf1_url, pdf1_path)
download_pdf(pdf2_url, pdf2_path)


'Book2.pdf'

# 將PDF切分為數個chunk

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
import os

def load_and_split_pdf(path):
  loader = PyPDFLoader(path)
  pages = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=20)
  docs = text_splitter.split_documents(pages)
  return docs

doc1_chunks = load_and_split_pdf(pdf1_path)
doc2_chunks = load_and_split_pdf(pdf2_path)


# 將文字訊息轉為向量資料庫形式

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.schema import Document # Import Document

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_db1 = Chroma.from_documents(doc1_chunks, embedding_model)
vector_db2 = Chroma.from_documents(doc2_chunks, embedding_model)

<ipython-input-4-f448eac2de6a>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 載入主要的LLM，Llama

In [ ]:
!pip install llama-cpp-python
!wget -O /content/tinyllama.gguf https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4067786 sha256=de37fa2ca6e1d1fb76383de2e808fd38949e13fb0bac5c05999cd94286e7d7a1
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python
--2025-05-14 04:53:04--  https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 65.8.243.92, 65.8.243.16, 65.8.243.90, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.92|:443... connected.
HTTP request sent, awaiting respons

# 設定langchain中使用的LLM和retriever

In [ ]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(
    model_path="/content/tinyllama.gguf",
    n_ctx=2048,
    temperature=0.7,
    max_tokens=512,
    top_p=0.9,
    verbose=True,
    stream=True
)

retriever1 = vector_db1.as_retriever(search_kwargs={"k": 15})
retriever2 = vector_db2.as_retriever(search_kwargs={"k": 15})

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! stream is not default parameter.
                stream was transferred to model_kwargs.
                Please confirm that stream is what you intended.
  if (await self.run_code(code, result,  async_=asy)):
llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from /content/tinyllama.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:           

# 根據問題產生回答並針對兩個不同的文件進行比較

In [ ]:
from langchain.chains import RetrievalQA
qa_chain1 = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever1,
    return_source_documents=True
)

qa_chain2 = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever2,
    return_source_documents=True
)

query = "historical background of the story"

answer1 = qa_chain1.invoke({"query": query})
answer2 = qa_chain2.invoke({"query": query})

compare_prompt = f"""
There are two documents about different stories that answer the same question: "{query}"

Document A says:
{answer1['result']}

Document B says:
{answer2['result']}

Please compare the two stories based on question: "{query}". List any differences between them in bullet points. Be sure don't mix them together.
"""

final_answer = llm.invoke(compare_prompt)

Llama.generate: 1 prefix-match hit, remaining 887 prompt tokens to eval
llama_perf_context_print:        load time =   69503.46 ms
llama_perf_context_print: prompt eval time =   71118.01 ms /   887 tokens (   80.18 ms per token,    12.47 tokens per second)
llama_perf_context_print:        eval time =   20494.58 ms /   148 runs   (  138.48 ms per token,     7.22 tokens per second)
llama_perf_context_print:       total time =   91825.07 ms /  1035 tokens
Llama.generate: 887 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   69503.46 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   24032.77 ms /   162 runs   (  148.35 ms per token,     6.74 tokens per second)
llama_perf_context_print:       total time =   24283.85 ms /   163 tokens
Llama.generate: 1 prefix-match hit, remaining 388 prompt tokens to eval
llama_perf_con

# 答案輸出

In [ ]:
print(final_answer)


Both documents include similar historical background information about the protagonist's life and death. However, Document B includes more details about his time at the convent, which may provide a different perspective on his emotional state. Additionally, Document A may have provided more information on the cultural context of the time period, while Document B may have focused more on the psychological effects of being sent to an unsuitable home.
Both documents also include descriptions of the protagonist's relationship with Maria, but Document A may focus more heavily on their love story and how it evolves over time, whereas Document B may offer a more nuanced portrayal of their dynamic that highlights the complexities of their feelings for each other.
Overall, both documents provide compelling insights into historical events that will add depth and complexity to your analysis.
